In [34]:
from torch.utils.data import Dataset
import torch
import numpy as np
import res.process_data.dire_and_coor as dc
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm.auto import tqdm

In [ ]:
# get self avoid polymer
self_avoid_directions = np.load('../../data/self_avoid/16/directions.npy')
self_avoid_coordinates = np.load('../../data/self_avoid/16/coorinates.npy')

In [ ]:
# get random polymer
random = np.random.randint(0, 4, size=(120000, 14, 1)) / 4
a = np.zeros((120000, 15, 1))
# make the first one is forward
for i in range(len(random)):
    a[i] = np.vstack(([0], random[i]))

In [ ]:
# remove self avoid data
random_coordinate = dc.direction_coordinate_three(a, 16)

In [ ]:
def check_fold_cross(coordinate):
    """
    check if a single polymer is folding or crossing
    """
    assert np.shape(coordinate) == (16, 2)
    folding_count = 0
    crossing_count = 0
    for i in range(0, len(coordinate)):
        temp = coordinate[i]
        if (temp == coordinate[:i]).all(axis=1).any():
            if np.array_equal(temp, coordinate[:i][-2]):
                folding_count += 1
            else:
                crossing_count += 1
    return [folding_count, crossing_count]

In [ ]:
c = []
for i in range(len(random_coordinate)):
    if check_fold_cross(random_coordinate[i]) != [0,0]:
        c.append(i) # self avoid

In [ ]:
j = 0
output = np.zeros((len(c), 14, 1))
for i in c:
    output[j] = random[i]
    j += 1

In [ ]:
# put two kinds of polymer together
final = np.concatenate([self_avoid_directions, output], axis=0)
np.shape(final)

In [ ]:
# from [0:100,000], we have the label [0,1]
# from [100,000:-1], we have the label [1,0]
label = np.zeros([len(final), 2])
for i in range(len(final)):
    if i < 99999:
        label[i] = np.array([0,1])
    else:
        label[i] = np.array([1,0])

In [ ]:
class tensor_dataset(Dataset):
    def __init__(self, input_tensor, input_label, row_dim, column_dim):
        self.input_tensor = input_tensor
        self.input_label = input_label
        self.row_dim = row_dim
        self.column_dim = column_dim

    def __len__(self):
        assert len(self.input_tensor) == len(self.input_label)
        return len(self.input_tensor)

    def __getitem__(self, index: int):
        tensor_id = index
        tensor = self.input_tensor[tensor_id]
        label = self.input_label[tensor_id]
        # convert the tensor from [x,y] to [1, x, y]
        tensor = torch.reshape(tensor, [1, self.row_dim, self.column_dim])
        # all label is 1(no label)
        tensor_label = label
        return tensor, tensor_label

In [31]:
lengthOfPolymer = np.shape(final)[1]

shuffle = True
num_worker = 0
pin_memory = True
batch_size = 8000
input_tensor = torch.Tensor(final)
input_label = torch.Tensor(label)
my_dataset = tensor_dataset(input_tensor, label, lengthOfPolymer, 1)
my_dataloader = DataLoader(dataset=my_dataset,
                           shuffle=shuffle,
                           batch_size=batch_size,
                           num_workers=num_worker,
                           pin_memory=pin_memory)

In [32]:
class Net(torch.nn.Module):
    """
    A two hidden-layer discriminative neural network
    """

    def __init__(self):
        super(Net, self).__init__()

        self.hidden0 = nn.Sequential(
            nn.Linear(14, 128),
            nn.LeakyReLU(0.2, inplace=True),
            # nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(128, 2),
            # torch.nn.Sigmoid()
        )

    def forward(self, z):
        z = self.hidden0(z)
        z = self.out(z)
        return z

In [36]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)
loss_func = torch.nn.CrossEntropyLoss()
for i in range (100):
    for data, labels in tqdm(my_dataloader):
        cur_batch_size = len(data)
        data = data.view(cur_batch_size, -1)
        out = net(data)
        #todo: loss function
        loss = loss_func(out, labels)
            # 喂给 net 训练数据 x, 输出分析值    # 计算两者的误差
        optimizer.zero_grad()   # 清空上一步的残余更新参数值
        loss.backward()         # 误差反向传播, 计算参数更新值
        optimizer.step()

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]